In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

__author__ = "Peter Nordin"
__license__ = "GPL"
__email__ = "peter.nordin@liu.se"

'''The purpose of this script is to allow easy calulation of the parasitic inductance and/or capacitance that
   will result from the choice of time step and material properties in Hopsan'''

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from TLMclasses import *
#import pandas as pd

In [ ]:
# Everything in this block is completely WRONG, ignore it

E=2.1e11
rho=7850
L=1
D=0.04*1
A=np.pi*D*D/4

mass_desired = A*L*rho
Zc_desired = np.sqrt(rho*A*A*E)
wavespeed = np.sqrt(E/rho)
Tdesired = L/wavespeed
stiffness_desired = E*A/L
n = 2                              # n = number of delay steps

template = "{0:20}{1:15}"
print(template.format('Desired mass:', str(mass_desired)))
print(template.format('Desired stiffness:', str(stiffness_desired)))
print(template.format('wavespeed:', str(wavespeed)))
print(template.format('Desired Zc:', str(Zc_desired)))
print(template.format('Tdesired:', str(Tdesired)))

ts = np.linspace(1e-6,5e-4,100)

Xdesired = [Tdesired*wavespeed+t*0 for t in ts]
Xpercived = [n*t*wavespeed for t in ts]
Mass_percived = [rho*A*n*t*wavespeed for t in ts]
ParasiticInd = [(n*t-Tdesired)*Zc_desired for t in ts] 
 

fig, axL = plt.subplots()
#plt.title('Length and parasitic mass realtive to simualtion timestep')
axL.plot(ts*1e6, Xpercived, label='Xpercived')
axL.plot(ts*1e6, Xdesired, label='Xdesired')
axL.set_ylabel('Length in meters')
axR = axL.twinx()
axR.plot(ts*1e6, ParasiticInd, color='red', label='Parsitic Mass')
axR.plot(ts*1e6, Mass_percived, color='y', label='Mass percived')
axR.set_ylabel('Mass in Kg')
axL.set_xlabel('Simulation Timestep in micro seconds')
axL.legend(loc='best')
axR.legend(loc='upper right')
plt.show()

In [ ]:
# Steady state Inductance test
# Preserve Char.impeedance, Get Parasitic Inductance (and capacitance, not listed here)
# Test is to apply constant Force, we should get reuslting velocity = F/m*tend 
force = 100
tend = 0.2

print('Tdesired: '+str(Tdesired)+' Massdesired: '+str(mass_desired)+' Velocitydesired: '+str(force/mass_desired*tend))
Ts = [Tdesired/n, Tdesired/n*2, Tdesired/n*4, Tdesired/n*8, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 5e-4, 1e-3, 1e-2, 1e-1, 1e-0]
Ts = sorted(Ts)

template = "{0:20} & {1:20} & {2:20} & {3:20} \\\\"
print(template.format("Time step", "Par. Inductance", "Percived Mass", "Integrated Velocity"))
for t in Ts:
    tp = TLMLineParameters(wavespeed, L, Zc_desired, mass_desired, stiffness_desired)
    tp.calcStuff(t,n)
    MassPercived = rho*tp.percived.length*A # Calc percived mass in a different way
    
    # v and v2 should be the same
    v = force/(tp.percived.ind)*tend
    v2 = force/(MassPercived)*tend
    print(template.format(str(t), str(tp.parasitic.ind), str(tp.percived.ind), str(v)))

In [ ]:
# Steady-state Capacitance test
# Preserve Char.impeedance, Get Parasitic Capacitance (and inductance, not listed here)
# Test is to apply constant Force, we should get a resulting displacement D = F/k = F*C (k = stiffness)

print('Tdesired: '+str(Tdesired)+' Massdesired: '+str(mass_desired)+' Desired Cap (1/k): '+str(1/stiffness_desired))
template = "{0:20}{1:20}{2:20}{3:20}{4:20}{5:20}"
print(template.format("Ts", "ParasiticCap", "CapacitancePercived", "StiffnessPercived", "D (parasitic cap)", "D (stiffnesspercived)"))
for t in Ts:
    tp = TLMLineParameters(wavespeed, L, Zc_desired, mass_desired, stiffness_desired)
    tp.calcStuff(t,n)
    Stiffness_percived = E*A/tp.percived.length                # Calculate based on percived length
    
    # d and d2 should be same
    d = force/tp.percived.stiffness
    d2 = force/Stiffness_percived
    print(template.format(str(t), str(tp.parasitic.cap), str(tp.percived.cap), str(tp.percived.stiffness), str(d), str(d2)))


In [ ]:
# Print again for LaTeX (same data)
template = "{0:20} & {1:20} & {2:20} & {3:20} \\\\"
print(template.format("Time step", "Par. Capacitance", "Percived Stiffness", "Steady-state Displacement"))
for t in Ts:
    tp = TLMLineParameters(wavespeed, L, Zc_desired, mass_desired, stiffness_desired)
    tp.calcStuff(t,n)

    d = force/tp.percived.stiffness
    print(template.format(str(t), str(tp.parasitic.cap), str(tp.percived.stiffness), str(d)))

In [ ]:
# Calculate wavespeeds

steelE = 210e9
steelRho = 7850
oilK = 1.6e9
oilRho = 890
l=1

steelSpeed = np.sqrt(steelE/steelRho)
oilSpeed = np.sqrt(oilK/oilRho)

print('Wavespeed in oil: '+str(oilSpeed)+' Tul: '+str(l/oilSpeed))
print('Wavespeed in steel: '+str(steelSpeed)+' Tul: '+str(l/steelSpeed))
